In [1]:
import os 

import torch
import torch.nn as nn 
import numpy as np
import matplotlib.pyplot as plt

from tqdm import tqdm

from helpers import load_experiment

In [2]:
dm, model = load_experiment("/home/danis/Projects/AlphaCaption/AutoConceptBottleneck/autoconcept/outputs/2023-05-10/05-22-25")

Fetching configuration...
Loading datamodule...


[nltk_data] Downloading package wordnet to /home/danis/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
/home/danis/anaconda3/envs/bottleneck/lib/python3.10/site-packages/torchtext/data/utils.py:105: UserWarning: Spacy model "en" could not be loaded, trying "en_core_web_sm" instead
  warnings.warn(
100%|██████████| 2700/2700 [00:00<00:00, 9701.65it/s]


Len of vocab:  54
Max len of caption:  16
Index for <pad>: [0]
Loading model


/home/danis/anaconda3/envs/bottleneck/lib/python3.10/site-packages/pytorch_lightning/utilities/parsing.py:262: UserWarning: Attribute 'criterion' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['criterion'])`.
  rank_zero_warn(


In [3]:
train_loader = dm.train_dataloader()
train_set = train_loader.dataset

In [4]:
# [red, green, blue, square, triangle, circle]

attribute_mapping = {
    0: [1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0
        ],
    1: [0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0
        ],
    2: [0, 0, 1, 1, 0, 0,  0, 0, 1, 0, 0, 0, 0, 0, 0
        ],
    3: [1, 0, 0, 0, 1, 0,   0, 0, 0, 1, 0, 0, 0, 0, 0
        ],
    4: [0, 1, 0, 0, 1, 0,   0, 0, 0, 0, 1, 0, 0, 0, 0
        ],
    5: [0, 0, 1, 0, 1, 0,   0, 0, 0, 0, 0, 1, 0, 0, 0
        ],
    6: [1, 0, 0, 0, 0, 1,   0, 0, 0, 0, 0, 0, 1, 0, 0
        ],
    7: [0, 1, 0, 0, 0, 1,   0, 0, 0, 0, 0, 0, 0, 1, 0
        ],
    8: [0, 0, 1, 0, 0, 1,   0, 0, 0, 0, 0, 0, 0, 0, 1
        ],
}

In [5]:
sample = train_set[50]
image = sample["image"]

image = image.unsqueeze(0).cuda()


# model.main.inference(image)[1].cpu().detach().numpy()[0]
model(image)["concept_probs"].cpu().detach().numpy()[0]

array([0.18115692, 0.32691154, 0.95352095, 0.11858784, 0.25531936,
       0.31551293, 0.04973926, 0.0362053 , 0.8385405 , 0.1468492 ],
      dtype=float32)

In [6]:
scores = list()

for feature_id in range(10):

    attribute_values = [[0, 0] for _ in range(len(attribute_mapping[0]))]

    for i in tqdm(range(len(train_set))):
        sample = train_set[i]
        
        idx = sample["target"].item()
        attributes = np.array(attribute_mapping[idx])

        image = sample["image"]
        image = image.unsqueeze(0).cuda()

        # feature = model.main.inference(image)[1].cpu().detach().numpy()[0][feature_id]
        feature = model(image)["concept_probs"].cpu().detach().numpy()[0][feature_id]

        for attr_id, attribute in enumerate(attributes):
            value_on = attribute * feature + (1 - attribute) * (1 - feature)
            attribute_values[attr_id][0] += value_on

            value_off = attribute * (1 - feature) + (1 - attribute) * feature
            attribute_values[attr_id][1] += value_off
    
    attribute_values_max = [max(p) for p in attribute_values]
    attribute_values_max = max(attribute_values_max)
    score = attribute_values_max / len(train_set)

    scores.append(score)

metric = np.array(scores).mean()

print(metric)

100%|██████████| 1845/1845 [00:36<00:00, 50.80it/s]

0.7329807396802224


Baseline: 0.7481238945043126 / 0.8002787673469156 / 

Our framework: 0.733698356318566 / 0.733698356318566 / 0.77456 (w/o textual, anneal)


In [7]:
scores # baseline

[0.7387194722921141,
 0.7676965905446812,
 0.7516099136302545,
 0.673259667055757,
 0.7868074841999264,
 0.6593413541048039,
 0.6775105591086029,
 0.6476065091748386,
 0.8292717628463658,
 0.5818808208090988]

In [8]:
scores # framework

[0.7387194722921141,
 0.7676965905446812,
 0.7516099136302545,
 0.673259667055757,
 0.7868074841999264,
 0.6593413541048039,
 0.6775105591086029,
 0.6476065091748386,
 0.8292717628463658,
 0.5818808208090988]